In [ ]:
!pip install faiss-cpu # Installs faiss for CPU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 61.2 MB/s eta 0:00:00


In [2]:
!pip install faiss-cpu # Installs faiss for CPU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 59.4 MB/s eta 0:00:00


In [7]:
# 📦 Import Dependencies
import openai
import getpass
import faiss
import numpy as np
import pandas as pd

# 🔐 Enter OpenAI API Key
openai.api_key = getpass.getpass("🔐 Enter your OpenAI API Key: ")

# 📄 Load Dataset
listings_df = pd.read_csv("/content/drive/MyDrive/USA Housing Dataset - USA Housing Dataset.csv")

# 📃 Prepare Full-Text Descriptions from Dataset
def generate_description(row):
    return (
        f"{int(row['bedrooms'])} bedroom(s), {row['bathrooms']} bathroom(s), "
        f"{int(row['sqft_living'])} sqft living space, "
        f"lot size: {int(row['sqft_lot'])} sqft, "
        f"{row['floors']} floor(s), "
        f"waterfront: {'Yes' if row['waterfront'] == 1 else 'No'}, "
        f"view score: {row['view']}, condition: {row['condition']}, "
        f"above ground: {int(row['sqft_above'])} sqft, "
        f"basement: {int(row['sqft_basement'])} sqft. "
        f"Built in {int(row['yr_built'])}, "
        f"renovated in {int(row['yr_renovated']) if row['yr_renovated'] != 0 else 'N/A'}. "
        f"Located at {row['street']}, {row['city']}, {row['statezip']}, {row['country']}. "
        f"Rent: {int(row['price'])} USD/month."
    )

listings_df["description"] = listings_df.apply(generate_description, axis=1)

# 🔢 Generate Embeddings
def get_embedding(text):
    response = openai.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return np.array(response.data[0].embedding)

# 🧠 Build Vector Store
embedding_dim = 1536
index = faiss.IndexFlatL2(embedding_dim)
listing_embeddings = np.vstack([get_embedding(desc) for desc in listings_df["description"]])
index.add(listing_embeddings)

# 🧠 System Role Instruction
conversation_history = [
    {
        "role": "system",
        "content": (
            "You are a helpful AI real estate assistant like Airbnb. "
            "Use the provided listings to respond clearly and concisely. "
            "Guide the user based on their preferences. Offer alternatives if no exact match is found."
        )
    }
]

# 🔄 RAG-Enabled Response Function with Token Safety & Contact Inquiry
def ask_ai_with_rag(user_input, special_req=None):
    global conversation_history

    user_vector = get_embedding(user_input).astype("float32").reshape(1, -1)
    top_k = 5
    distances, indices = index.search(user_vector, top_k)

    context = "\n".join(listings_df.iloc[idx]["description"] for idx in indices[0])
    if special_req:
        user_input += f" Also, consider this special preference: {special_req}"

    prompt = (
        f"User: {user_input}\n\n"
        f"Relevant Listings:\n{context}\n\n"
        "Provide a helpful, professional, and personalized response. "
        "If listings do not match, offer options like adjusting budget or location. "
        "Also, ask if they want to provide email and WhatsApp for future updates. "
        "Ask how many parking spaces they need if relevant."
    )

    # Limit context size by trimming conversation history
    conversation_history = conversation_history[:1]  # Keep only system prompt
    conversation_history.append({"role": "user", "content": prompt})

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=conversation_history,
        temperature=0.4,
    )

    reply = response.choices[0].message.content
    conversation_history.append({"role": "assistant", "content": reply})

    return reply

# 👤 Classify User Type
def classify_user_type(user_input: str) -> str:
    prompt = (
        "Classify the type of user based on the following message:\n\n"
        f"\"{user_input}\"\n\n"
        "Options: [tourist, investor, student, family, unknown]. "
        "Return only one of these options."
    )
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    result = response.choices[0].message.content.strip().lower()
    return result if result in ['tourist', 'investor', 'student', 'family'] else 'unknown'

# 🚀 Main Chat Loop
print("🏡 Welcome to the AI Real Estate Assistant with RAG!\n")

# Step 1: Initial Query
initial_message = input("📝 Describe what you're looking for (e.g., '3-bedroom in Seattle under 2500 USD near waterfront'): ")
user_type = classify_user_type(initial_message)
print(f"👤 Detected User Type: {user_type}\n")

special_choice = input("⭐ Any special preferences for this search? (press Enter to skip): ")
response = ask_ai_with_rag(initial_message, special_req=special_choice)
print("\n🤖 AI Assistant Response:\n")
print(response)

# Step 2: Continuous Conversation Loop
while True:
    follow_up = input("\n💬 Ask more or refine search (type 'exit' to quit): ")
    if follow_up.strip().lower() in ['exit', 'quit', 'no']:
        print("👋 Thank you for using the AI Real Estate Assistant. Goodbye!")
        break

    special_choice = input("⭐ Any additional special preferences? (press Enter to skip): ")
    response = ask_ai_with_rag(follow_up, special_req=special_choice)
    print("\n🤖 AI Assistant Follow-up Response:\n")
    print(response)

    # 🔔 If no match found, ask for contact
    if any(kw in response.lower() for kw in ["no match", "not available", "none of these"]):
        email = input("📧 Could you share your email to receive updates? (optional): ")
        whatsapp = input("📱 WhatsApp number for updates? (optional): ")
        parking = input("🚗 How many parking spots do you need? (optional): ")
        if email or whatsapp:
            print("✅ Thank you! You will receive updates via your provided contact.")


🔐 Enter your OpenAI API Key: ··········
🏡 Welcome to the AI Real Estate Assistant with RAG!

📝 Describe what you're looking for (e.g., '3-bedroom in Seattle under 2500 USD near waterfront'): 3 bed room under 10000k..with 3 parking
👤 Detected User Type: investor

⭐ Any special preferences for this search? (press Enter to skip): parking

🤖 AI Assistant Response:

I'm sorry, but it seems there might be a misunderstanding with the budget you've specified. The listings provided are all above your budget of 10,000 USD per month. The lowest rent among these listings is 385,000 USD per month. 

Additionally, none of these listings specify the number of parking spaces available. Could you please confirm the number of parking spaces you require? 

If you're flexible with your budget or location, I could help find other options that meet your requirements. Also, if you could provide your email and WhatsApp details, I can send you updates on new listings that match your preferences.

💬 Ask more or